In [ ]:
import os
import numpy as np
import albumentations as A
import cv2
import torch as T
from torch.utils.data import Dataset
from PIL import Image
from tqdm import tqdm

In [ ]:
albumentations_transform =  A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.Blur(p=0.3),
    A.OpticalDistortion(p=0.2),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.2),
    A.ShiftScaleRotate(rotate_limit=2.5, p=0.5,),
], bbox_params=A.BboxParams(format='yolo', label_fields=['category_ids']))

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_folder, label_folder, transform=None):
        self.image_folder = image_folder
        self.label_folder = label_folder
        self.image_names = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.image_folder, img_name)
        label_path = os.path.join(self.label_folder, img_name.replace('.jpg', '.txt'))

        image = np.array(Image.open(img_path).convert("RGB"))

        with open(label_path, 'r') as file:
            bboxes = []
            category_ids = []
            for line in file:
                parts = line.strip().split()
                category_ids.append(int(parts[0]))
                bboxes.append([float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])])

        if self.transform:
            augmented = self.transform(image=image, bboxes=bboxes, category_ids=category_ids)
            image = augmented['image']
            bboxes = augmented['bboxes']
            category_ids = augmented['category_ids']
        
        image = T.from_numpy(image.transpose((2, 0, 1))).float() / 255.0
        return image, bboxes, category_ids, img_name

image_folder = "C:/Users/yhd06/OneDrive/Desktop/iwaz/pothole_data/pothole_data/images"
label_folder = "C:/Users/yhd06/OneDrive/Desktop/iwaz/pothole_data/pothole_data/labels"
dataset = CustomDataset(image_folder, label_folder, transform=albumentations_transform)

In [ ]:
def save_augmented_data(dataset, output_image_folder, output_label_folder, num_augmentations):
    if not os.path.exists(output_image_folder):
        os.makedirs(output_image_folder)
    if not os.path.exists(output_label_folder):
        os.makedirs(output_label_folder)
    
    for i in tqdm(range(len(dataset))):
        image, bboxes, category_ids, img_name = dataset[i]
        image = image.numpy().transpose((1, 2, 0)) * 255
        image = image.astype(np.uint8)

        for j in range(num_augmentations):
            augmented = dataset.transform(image=image, bboxes=bboxes, category_ids=category_ids)
            aug_image = augmented['image']
            aug_bboxes = augmented['bboxes']
            aug_category_ids = augmented['category_ids']
            
            aug_image_name = f"{img_name.replace('.jpg', '')}_aug_{j}.jpg"
            aug_image_path = os.path.join(output_image_folder, aug_image_name)
            cv2.imwrite(aug_image_path, cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR))

            aug_label_path = os.path.join(output_label_folder, f"{img_name.replace('.jpg', '')}_aug_{j}.txt")
            with open(aug_label_path, 'w') as file:
                for bbox, category_id in zip(aug_bboxes, aug_category_ids):
                    file.write(f"{category_id} {' '.join(map(str, bbox))}\n")

output_image_folder = "C:/Users/yhd06/OneDrive/Desktop/iwaz/pothole_data/output_pothole_dataset/images"
output_label_folder = "C:/Users/yhd06/OneDrive/Desktop/iwaz/pothole_data/output_pothole_dataset/labels"

save_augmented_data(dataset, output_image_folder, output_label_folder, num_augmentations=20)


In [ ]:
image_folder = "C:/Users/yhd06/OneDrive/Desktop/iwaz/pothole_data/output_pothole_dataset/images"
label_folder = "C:/Users/yhd06/OneDrive/Desktop/iwaz/pothole_data/output_pothole_dataset/labels"
output_image_folder = "C:/Users/yhd06/OneDrive/Desktop/iwaz/pothole_data/output_images"

def draw_bbox(image_path, label_path, output_image_folder):
    image = cv2.imread(image_path)
    if image is None:
        print(f"이미지를 불러오는 것을 실패하였습니다: {image_path}")
        return
    height, width, _ = image.shape

    with open(label_path, 'r') as f:
        labels = f.readlines()

    for label in labels:
        class_id, x_center, y_center, bbox_width, bbox_height = map(float, label.split())

        x_center = int(x_center * width)
        y_center = int(y_center * height)
        bbox_width = int(bbox_width * width)
        bbox_height = int(bbox_height * height)

        x1 = int(x_center - bbox_width / 2)
        y1 = int(y_center - bbox_height / 2)
        x2 = int(x_center + bbox_width / 2)
        y2 = int(y_center + bbox_height / 2)

        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    os.makedirs(output_image_folder, exist_ok=True)

    output_image_path = os.path.join(output_image_folder, os.path.basename(image_path))

    cv2.imwrite(output_image_path, image)
    print(f"바운딩 박스가 있는 저장된 이미지: {output_image_path}")

for image_filename in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_filename)
    label_path = os.path.join(label_folder, os.path.splitext(image_filename)[0] + '.txt')

    if os.path.exists(label_path):
        draw_bbox(image_path, label_path, output_image_folder)
    else:
        print(f"해당 이미지의 라벨 파일을 찾을 수 없음: {image_filename}")